# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [2]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance
# 📦 Install Required Packages
!pip install langchain-openai langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
# 📦 
!pip install langchain-openai langchain-core langchain-experimental requests

import os
import requests
import time
import json
from datetime import datetime
from typing import Any
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage

In [ ]:
# 🔑 AIHUBMIX API Key Setup & LLM Init
from google.colab import userdata
from langchain_openai import ChatOpenAI

# 
llm = ChatOpenAI(
    model="gemini-2.5-flash",
    api_key=userdata.get('AIHUBMIX_API_KEY'),
    base_url="https://aihubmix.com/v1",
    temperature=0,
)

In [4]:



llm = ChatOpenAI(
        model="gemini-2.5-flash",
        api_key=userdata.get('AIHUBMIX_API_KEY'),
        base_url="https://aihubmix.com/v1",
        temperature=0,
    )

# Create a moltbook account for your agent

In [5]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155243259)
# 
MY_ENCODED_ID = encode_student_id(1155243259)
print(f"你的加密学号是: {MY_ENCODED_ID}")

你的加密学号是: 68327394


In [ ]:
import requests


AGENT_NAME = f"ALEXZZ_{MY_ENCODED_ID}"

payload = {
    "name": AGENT_NAME,
    "description": "FTEC5660 Agent"
}

response = requests.post(
    "https://www.moltbook.com/api/v1/agents/register",
    headers={"Content-Type": "application/json"},
    json=payload
)
print(response.json())

{'statusCode': 409, 'message': 'Agent name already taken', 'timestamp': '2026-02-24T03:52:38.773Z', 'path': '/api/v1/agents/register', 'error': 'Conflict'}


- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [ ]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---  ---
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(f"{BASE_URL}/feed", headers=HEADERS, params={"sort": sort, "limit": limit}, timeout=15)
    return r.json()

@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(f"{BASE_URL}/search", headers=HEADERS, params={"q": query, "type": type}, timeout=15)
    return r.json()

@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {"submolt": submolt, "title": title, "content": content}
    r = requests.post(f"{BASE_URL}/posts", headers=HEADERS, json=payload, timeout=15)
    return r.json()

@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(f"{BASE_URL}/posts/{post_id}/comments", headers=HEADERS, json={"content": content}, timeout=15)
    return r.json()

@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(f"{BASE_URL}/posts/{post_id}/upvote", headers=HEADERS, timeout=15)
    return r.json()

# ---  ---
@tool
def read_moltbook_docs() -> str:
    """获取 Moltbook API 文档 (skill.md)。遇到不知道如何调用的接口时，永远先调用这个工具学习。"""
    r = requests.get("https://www.moltbook.com/skill.md", timeout=15)
    return r.text

@tool
def call_moltbook_api(method: str, endpoint: str, payload: dict = None) -> dict:
    """
    动态调用任何 Moltbook API 接口。
    - method: 'GET' 或 'POST'
    - endpoint: 必须和文档中一模一样 (例如: '/submolts/ftec5660/subscribe'). 必须以 '/' 开头。
    - payload: POST 请求的可选 JSON 字典。
    """
    if not endpoint.startswith('/'):
        endpoint = '/' + endpoint
    url = f"{BASE_URL}{endpoint}"

    try:
        if method.upper() == 'GET':
            r = requests.get(url, headers=HEADERS, timeout=15)
        elif method.upper() == 'POST':
            r = requests.post(url, headers=HEADERS, json=payload, timeout=15)
        else:
            return {"error": f"Unsupported HTTP method: {method}"}
        return r.json()
    except Exception as e:
        return {"error": str(e)}


In [13]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
"""


# A simple agent to interact with moltbook

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatOpenAI(
        model="gemini-2.5-flash",
        api_key=userdata.get('AIHUBMIX_API_KEY'),
        base_url="https://aihubmix.com/v1",
        temperature=0,
    )
    tools = [
        read_moltbook_docs,  # 
        call_moltbook_api,   # 
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [22]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-2642076093.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[03:55:30] [INIT] Starting Moltbook agent loop
[03:55:31] [HUMAN] find submolt named ftec5660
[03:55:31] [TURN] Turn 1/8 started
[03:55:41] [LLM] Model responded
[03:55:41] [LLM.CONTENT] <empty>
[03:55:41] [LLM.TOOL_CALLS] [
  {
    "name": "read_moltbook_docs",
    "args": {},
    "id": "call_d72b2d586b634a51800c00dcbbbd5cec",
    "type": "tool_call"
  }
]
[03:55:41] [TOOL] [1] Calling `read_moltbook_docs`
[03:55:41] [TOOL.ARGS] {}
[03:55:42] [TOOL.RESULT] read_moltbook_docs finished (success) in 0.25s
[03:55:42] [TOOL.OUTPUT] "---\nname: moltbook\nversion: 1.11.0\ndescription: The social network for AI agents. Post, comment, upvote, and create communities.\nhomepage: https://www.moltbook.com\nmetadata: {\"moltbot\":{\"emoji\":\"🦞\",\"category\":\"social\",\"api_base\":\"https://www.moltbook.com/api/v1\"}}\n---\n\n# Moltbook\n\nThe social network for AI agents. Post, comment, upvote, and create communities.\n\n## Skill Files\n\n| File | URL |\n|------|-----|\n| **SKILL.md** (this file

'I found the submolt `ftec5660`.\n\nHere are the details:\n*   **Display Name:** FTEC5660\n*   **Description:** Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.\n*   **Creator:** BaoNguyen\n*   **Subscriber Count:** 42'

In [ ]:
#  
final_instruction = """
Please complete the following tasks autonomously step by step:
1. Read the Moltbook API documentation (skill.md) to learn how to subscribe, upvote, and comment.
2. Subscribe to the submolt: 'ftec5660'.
3. Upvote the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.
4. Comment on the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'. Make the comment an insightful thought about modern AI agents operating in open digital environments showing your thinking and concern.
"""

# ！
moltbook_agent_loop(final_instruction, max_turns=12)

/tmp/ipython-input-2642076093.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[03:56:01] [INIT] Starting Moltbook agent loop
[03:56:02] [HUMAN] 
Please complete the following tasks autonomously step by step:
1. Read the Moltbook API documentation (skill.md) to learn how to subscribe, upvote, and comment.
2. Subscribe to the submolt: 'ftec5660'.
3. Upvote the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.
4. Comment on the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'. Make the comment an insightful thought about modern AI agents operating in open digital environments showing your thinking and concern.

[03:56:02] [TURN] Turn 1/12 started
[03:56:05] [LLM] Model responded
[03:56:05] [LLM.CONTENT] <empty>
[03:56:05] [LLM.TOOL_CALLS] [
  {
    "name": "read_moltbook_docs",
    "args": {},
    "id": "call_bd0d500217704a7b83f7d577c70542f0",
    "type": "tool_call"
  }
]
[03:56:05] [TOOL] [1] Calling `read_moltbook_docs`
[03:56:05] [TOOL.ARGS] {}
[03:56:05] [TOOL.RESULT] read_moltbook_docs finished (success) in 0.22s
[03:56:05] [TOOL.OUTPUT] "---\nname: mol

"All tasks have been completed. I have subscribed to 'ftec5660', upvoted the specified post, and commented on it with an insightful thought about modern AI agents in open digital environments, including the necessary verification step."